In [ ]:
#Dataset Creation of Election and Food Reviews CSV
import os
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import StratifiedKFold
import preprocessor as p
from nltk import PorterStemmer 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import string
import pandas as pd
from stw import SupervisedTermWeightingWTransformer
from numpy import array
import ast
from nltk.corpus import stopwords 

In [ ]:
def preprocessing(line):
    line=p.clean(line)
    line = line.lower()
    line = line.split()
    for i in range(len(line)):
        lemmatizing_token=lemmatizer.lemmatize(line[i])
        line[i]=lemmatizing_token
    translation = str.maketrans("","", string.punctuation);
    for i in range(len(line)):
        line[i]=line[i].translate(translation)
    
    stop_words = set(stopwords.words('english'))
    line=[token for token in line if not token in stop_words]

    line=[token for token in line if token.isalpha()]
    line=[token for token in line if len(token)>2]
    line = " ".join(line)
    return line

In [ ]:
def make_Corpus(root_dir,polarity_dirs):
    corpus = []
    for polarity_dir in polarity_dirs:
        reviews = [os.path.join(polarity_dir,f) for f in os.listdir(polarity_dir)]
        for review in reviews:
            doc_string = "";
            with open(review) as rev:
                print(review)
                for line in rev:
                    doc_string = doc_string + line
                    doc_string+=" "
            if not corpus:
                corpus = [doc_string]
            else:
                corpus.append(doc_string)
    return corpus

root_dir = 'Elections/neg/'
pos_corpus = make_Corpus(root_dir,['Elections/neg/'])

root_dir = 'Elections/pos/'
neg_corpus = make_Corpus(root_dir,['Elections/pos/'])


In [ ]:
df=pd.read_csv("Reviews.csv")
df.dropna(inplace=True)
df[df['Score'] != 3]
df['Positivity'] = np.where(df['Score'] > 3, 1, 0)
df.head()

pos_corpus=[]
neg_corpus=[]
pos=0
neg=0
for i in range(len(df)):
    if df.iloc[i][10] == 1:
        pos_corpus.append(df.iloc[i][8])
        pos+=1
    elif df.iloc[i][10] == 0:
        neg_corpus.append(df.iloc[i][8])
        neg+=1
print("Done")

def make_CSV_corpus(neg_corpus,pos_corpus):
    corpus = []    
    file_counter=0
    for line in neg_corpus:
        line = preprocessing(line)
        the_file = open("Food_Reviews/pos/file_"+str(file_counter)+".txt", 'w+')
        the_file.write(line)
        the_file.close()
        file_counter+=1
        corpus.append(line)
    
    for line in pos_corpus:
        line = preprocessing(line)
        the_file = open("Food_Reviews/neg/file_"+str(file_counter)+".txt", 'w+')
        the_file.write(line)
        the_file.close()
        file_counter+=1
        corpus.append(line)
    return corpus

corpus = make_CSV_corpus(neg_corpus,pos_corpus)